In [2]:
import pandas as pd
from pathlib import PurePath

df = pd.read_csv(PurePath("..") / '6_to_8am.csv', sep=';', header=None, names=['vehicle_id', 'time', 'x', 'y', 'duration'], index_col=False)
df.head(20)

,vehicle_id,time,x,y,duration
0,0,24180,60,50,1
1,0,24190,60,49,2
2,0,24210,60,48,1
3,0,24220,60,47,3
4,0,24250,59,46,2
5,0,24270,58,46,2
6,0,24290,57,46,1
7,0,24300,56,46,1
8,0,24310,56,45,1
9,0,24320,55,45,2


In [3]:
# convert duration to seconds
df['duration_s'] = df['duration'] * 10

df['cell_id'] = df[['x', 'y']].astype(str).agg('_'.join, axis=1)

df.sort_values(['vehicle_id', 'time'], inplace=True)

df['vehicle_start_time'] = df.groupby('vehicle_id')['time'].transform('min')

df['elapsed_time'] = df['time'] - df['vehicle_start_time']

tau = 30
df_within_tau = df[df['elapsed_time'] <= tau]

df_within_tau = df_within_tau[['vehicle_id', 'cell_id']].drop_duplicates()

cells_groups = df_within_tau.groupby('vehicle_id')['cell_id'].apply(list)

with open('cells_per_vehicle.csv', 'w') as f:
    for cell in cells_groups:
        f.write(';'.join(cell) + '\n')